In [ ]:
! pip install sklearn

# Imports

In [ ]:
from fastai.text import *

In [ ]:
import sys

sys.path.append("../../")
from eval.exp.nb_evaluation import *

sys.path.append("../../../")
from src.proc.exp.nb_proc import *
from src.prep.exp.nb_prep import *


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# setup paths and model type
model_path = Path("/tf/data/models")
data_path  = Path("/tf/data/datasets")

task_type = "merged"

# Load Data

In [ ]:
df_trn, df_val, df_tst = read_data(data_path/task_type)

In [ ]:
bs = 8

Percentage of data to be used: sample 

In [ ]:
data = gen_lm_data(df_trn, df_val, task_type, data_path, bs = bs)
data.save(task_type + '/data_lm_100pct.pkl')

In [ ]:
data = load_data(data_path/task_type, 'data_lm_100pct.pkl', bs=bs)

In [ ]:
len(data.train_ds), len(data.valid_ds)

# Model Setup

In [ ]:
pretrained = False

In [ ]:
# amit experiments
learn = language_model_learner(data, AWD_LSTM,
                               drop_mult = 0.3, pretrained = pretrained,
                               metrics=[accuracy])

In [ ]:
learn.lr_find()
learn.recorder.plot()

# Model Training

In [ ]:
# Set hyperparameters
max_lr = 1e-2
moms = (0.5, .75)
pct_strt = 0.02
a_epochs = 10

In [ ]:
callbacks = [
    callbacks.SaveModelCallback(
        learn, every='improvement',
        monitor='valid_loss', name='awd_lstm_save_model'
    )
]

In [ ]:
#amit experiments
learn.fit_one_cycle(a_epochs, max_lr, callbacks = callbacks)

In [ ]:
learn.load(model_path/'awd_lstm_do_30pct')

In [ ]:
learn = load_model(df_trn, file = model_path/'awd_lstm_do_30pc.pkl')

In [ ]:
figure_plot = learn.recorder.plot_losses(return_fig=True)

In [ ]:
figure_plot.savefig(fname="awd_lstm_plot_losses.png", format='png')

In [ ]:
from PIL import Image
Image.open('/tf/main/nbs/mdling/awd_lstm/awd_lstm_plot_losses.png')

# Model Evaluation

In [ ]:
# import
sys.path.append("/tf/main/nbs/")
from eval.exp.nb_evaluation import *

### Vulnerability Classification

In [ ]:
task_type = "buggy"
vuln_trn, vuln_val, vuln_tst = read_data(data_path/task_type)

In [ ]:
acc, prec, recal = eval_vuln(learn, vuln_val)

In [ ]:
acc, prec, recal

In [ ]:
import sentencepiece as spm
sp = spm.SentencePieceProcessor()
sp.Load(str(data_path/"merged/model.model"))

In [ ]:
TEXT = df_val["query"][500]
N_WORDS = 200
N_SENTENCES = 2

In [ ]:
print(TEXT)
df_val["res"][500]

In [ ]:
print(sp.DecodePieces(learn.predict(TEXT, 100, temperature=0.75).split(" ")))

# Evaluation

In [ ]:
learn.predict??

In [ ]:
# Grabs entire model's response up until special xxbos token,
# i.e. once model begins a new sentence we consider the model finished with its answer.
def get_res(mdl, inpt, n_toks = 1_000):
    res = mdl.predict(inpt, n_toks, temperature=0.75).split(" ")
    res = sp.DecodePieces(res).split(" ")
    try:
        end_res = res.index("xxbos")
    except:
        end_res = len(res) - 1
    
    res = " ".join(res[:end_res])[len(inpt.replace(" ", '')):]
    
    return res

In [ ]:
res = get_res(learn, "public static void main() {return;}<$bug$>", n_toks = 10)
res # [0:len("public static void main() {return;}<$bug$>")]

In [ ]:
#export
def eval_txt(mdl, ds):
    b1, b2, b3, b4 = [], [], [], []
    meteor = []
    rouge_l = []
    for inpt, lbl in zip(ds["query"], ds["res"]):
        tok_len = len(sp.EncodeAsPieces(inpt))
        if tok_len > 1024:
            continue
            
        pred = get_res(mdl, inpt, n_toks = 10)
        # bleu 1-4
        b1.append(eval_bleu1([lbl], pred))
        b2.append(eval_bleu2([lbl], pred))
        b3.append(eval_bleu3([lbl], pred))
        b4.append(eval_bleu4([lbl], pred))
        
        # meteor
        meteor.append(eval_meteor([lbl], pred))
        
        # rouge
        rouge_l.append(eval_rougeL([lbl], pred))
        
    return b1, b2, b3, b4, meteor, rouge_l

In [ ]:
# Evaluation metrics for vulnerability detection - Accuracy, Precision, Recall
def eval_vuln(mdl, tst):
    tps, tns, fps, fns = 0, 0, 0, 0
    tot = 0
    for inpt, lbl in zip(tst["query"], tst["res"]):
        tok_len = len(sp.EncodeAsPieces(inpt))
        if tok_len > 1024:
#             print("Skipping because size is too big", tok_len)
            continue
        pred = get_res(mdl, inpt, n_toks = 10)
        if lbl == "yes":
            if pred == lbl:
                tps += 1
            else: fns += 1
        else:
            if pred == lbl:
                tns += 1
            else: fps += 1
                
        tot += 1
        torch.cuda.empty_cache()
            
    acc   = (tps + tns) / tot
    prec  = tps / (tps + fps) if (tps + fps) != 0 else 0.
    recal = tps / (tps + fns) if (tps + fns) != 0 else 0.
    
    return acc, prec, recal

In [ ]:
acc, prec, recal = eval_vuln(learn, df_val[:1000])

In [ ]:
acc, prec, recal

In [ ]:
len(df_val[:10])

In [ ]:
import torch
torch.cuda.empty_cache()